# YeaZ output evaluation

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files 
import matplotlib.pyplot as plt
%matplotlib notebook

try:
  from download import download
except ImportError:
  !pip3 install download
  from download import download

try:
    from yeastcells import data, model, clustering, features, visualize, evaluation
except ImportError:
    !test -e yeastcells-detection-maskrcnn || git clone https://github.com/ymzayek/yeastcells-detection-maskrcnn.git
    !cd yeastcells-detection-maskrcnn; git pull origin main
    !pip3 install ./yeastcells-detection-maskrcnn
    from yeastcells import data, model, clustering, features, visualize, evaluation

### Load YeaZ segmented masks

In [ ]:
pipeline_path = f'/content/yeastcells-detection-maskrcnn'

download(
    'https://datascience.web.rug.nl/models/yeast-cells/mask-rcnn/v1/YeaZ_results.zip',
    f'{pipeline_path}/YeaZ_results/YeaZ_results.zip'
)

download(
    'https://datascience.web.rug.nl/YIT-Benchmark2.zip', 
    f'{pipeline_path}/YIT-Benchmark2/YIT-Benchmark2.zip'
)

!cd '{pipeline_path}' && unzip 'YIT-Benchmark2/YIT-Benchmark2.zip' -d YIT-Benchmark2
!cd '{pipeline_path}' && unzip 'YeaZ_results/YeaZ_results.zip' -d YeaZ_results

In [ ]:
testset = 'TestSet1' 
masks = data.read_tiff_mask(f'{pipeline_path}/YeaZ_results/YeaZ_masks_{testset}.tiff')
np.shape(masks)

(60, 512, 512)

### Extract labels from masks

In [ ]:
labels, labels_grouped, coordinates, instances = features.extract_labels(masks)

### Load ground truth data and reformat prediction outputs into array

In [ ]:
seg_path = f'{pipeline_path}/YIT-Benchmark2/{testset}/GroundTruth/GroundTruth_Segmentation.csv'
track_path = f'{pipeline_path}/YIT-Benchmark2/{testset}/GroundTruth/GroundTruth_Tracking.csv'
gt_s, gt_t = data.get_gt_yit(seg_path, track_path)
pred_s, pred_t = data.get_pred_yeaz(labels, labels_grouped, coordinates)

### Match predictions to ground truth

In [ ]:
results_s = evaluation.get_seg_performance(pred_s, gt_s, instances, pipeline='YeaZ')
results_t = evaluation.get_track_performance(pred_t, gt_t, instances, pipeline='YeaZ')

In [ ]:
print(results_s)
print(results_t)

{'tp': 1115, 'fp': 553, 'fn': 51, 'join': 7, 'split': 0}
{'tp': 1102, 'fp': 517, 'fn': 37, 'join': 2, 'split': 2}


In [ ]:
metrics_s = evaluation.calculate_metrics(results_s, pred_s, gt_s)
metrics_t = evaluation.calculate_metrics(results_t, pred_t, gt_t)

In [ ]:
print(metrics_s)
print(metrics_t)

{'F1-score': 0.7868736767819337, 'Accuracy': 0.6486329261198371, 'Precision': 0.6684652278177458, 'Recall': 0.9562607204116638}
{'F1-score': 0.7991298042059464, 'Accuracy': 0.6654589371980676, 'Precision': 0.6806670784434836, 'Recall': 0.9675153643546971}
